In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve,train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [36]:
data = pd.read_csv("N://credit_default-master/UCI_Credit_Card.csv")

In [37]:
data['bill_0count'] = data[['BILL_AMT1','BILL_AMT2',
                    'BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].apply(lambda x : x.value_counts().get(0,0),axis=1)

In [38]:
data['pay_0count'] = data[['PAY_AMT1','PAY_AMT2',
                    'PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']].apply(lambda x : x.value_counts().get(0,0),axis=1)

In [39]:
data.eval('pay_sub_bill = pay_0count - bill_0count', inplace=True)

In [40]:
data.drop(['ID','pay_0count','bill_0count'], inplace=True, axis=1)
target = data['default.payment.next.month']
columns = data.columns.tolist()
columns.remove('default.payment.next.month')
features = data[columns]

In [48]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder(sparse=False)
sex = OHE.fit_transform(features[['SEX']])
marriage = OHE.fit_transform(features[['MARRIAGE']])
education = OHE.fit_transform(features[['EDUCATION']])

cat_variables = np.hstack((sex, marriage, education))

cat_var_names = ['SEX','MARRIAGE', 'EDUCATION']

num_variables = features.drop(cat_var_names, axis=1)

final_features = np.hstack((cat_variables,num_variables))

G:\Users\gggg\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
G:\Users\gggg\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used

In [49]:
train_x, test_x, train_y, test_y = train_test_split(final_features, target, test_size=0.20, stratify = target, random_state = 1)

In [50]:
classifiers = [
    SVC(random_state = 1, kernel = 'rbf'),
    #RandomForestClassifier(random_state = 1, criterion = 'gini'),
    #KNeighborsClassifier(metric = 'minkowski'),
]

In [51]:
classifier_names = [
            'svc', 
            #'randomforestclassifier',
            #'kneighborsclassifier',
]

In [52]:
classifier_param_grid = [
            {'svc__C':[1], 'svc__gamma':[0.01]},
            #{'randomforestclassifier__n_estimators':[3,5,6]} ,
            #{'kneighborsclassifier__n_neighbors':[4,6,8]},
]

In [53]:
def GridSearchCV_work(pipeline, train_x, train_y, test_x, test_y, param_grid, score = 'accuracy'):
    response = {}
    gridsearch = GridSearchCV(estimator = pipeline, param_grid = param_grid, scoring = score)
    # 寻找最优的参数 和最优的准确率分数
    search = gridsearch.fit(train_x, train_y)
    print("GridSearch最优参数：", search.best_params_)
    print("GridSearch最优分数： %0.4lf" %search.best_score_)
    predict_y = gridsearch.predict(test_x)
    print("准确率 %0.4lf" %accuracy_score(test_y, predict_y))
    response['predict_y'] = predict_y
    response['accuracy_score'] = accuracy_score(test_y,predict_y)
    return response

In [54]:
for model, model_name, model_param_grid in zip(classifiers, classifier_names, classifier_param_grid):
    pipeline = Pipeline([
            ('scaler', StandardScaler()),
            (model_name, model)
    ])
    result = GridSearchCV_work(pipeline, train_x, train_y, test_x, test_y, model_param_grid , score = 'accuracy')

G:\Users\gggg\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearch最优参数： {'svc__C': 1, 'svc__gamma': 0.01}
GridSearch最优分数： 0.8178
准确率 0.8220
